In [118]:
year = '57'
pdf_filename = 'data/pdf-text' + year + '.txt'
major_filename = 'data/majors-adm' + year + '.csv'
output_filename = 'data/major-scores-adm' + year + '.csv'

In [119]:
lines = open(pdf_filename).readlines()
print(lines[:2])

['www.cuas.or.th ตารางแสดงสถตจานวนผสมคร จานวนรบ จานวนผผานฯ และคะแนนสงสด-ตาสด ของการคดเลอกบคคลเขาศกษาในสถาบนอดมศกษาระบบกลาง (Admissions) ประจาปการศกษา 2557\n', '1.จฬาลงกรณมหาวทยาลย\n']


In [120]:
import re
major_number_pattern = r'^\d\d\d\d$'
major_number = re.compile(major_number_pattern)

In [121]:
old_key='xxxx'
old_items = []
pre_results = []
for l in lines:
    for item in l.split():        
        if major_number.match(item) and (old_items==[] or ('มหา' not in old_items[-1])):
            pre_results.append((old_key, old_items))
            old_items = []
            old_key = item
        else:
            old_items.append(item)
pre_results.append((old_key, old_items))
print(pre_results[:100])

[('xxxx', ['www.cuas.or.th', 'ตารางแสดงสถตจานวนผสมคร', 'จานวนรบ', 'จานวนผผานฯ', 'และคะแนนสงสด-ตาสด', 'ของการคดเลอกบคคลเขาศกษาในสถาบนอดมศกษาระบบกลาง', '(Admissions)', 'ประจาปการศกษา']), ('2557', ['1.จฬาลงกรณมหาวทยาลย', 'รหส', 'คณะ/สาขา', 'จานวน', 'คะแนน', 'สมคร', 'รบ', 'ผาน', 'สงสด', 'ตาสด', 'คาเฉลย', 'SD']), ('0001', ['คณะทนตแพทยศาสตร', '329', '29', '29', '23,640.7000', '22,437.4000', '22,810.9724', '337.2389']), ('0002', ['คณะเภสชศาสตร\uf70eสาขาวชาเภสชศาสตร', '553', '100', '100', '23,024.4500', '20,221.5500', '20,764.8900', '493.8322']), ('0003', ['คณะเภสชศาสตร\uf70eสาขาวชาการบรบาลทางเภสชกรรม', '454', '90', '90', '21,661.2500', '20,129.4500', '20,600.7872', '396.1231']), ('0004', ['คณะสตวแพทยศาสตร', '539', '160', '160', '24,077.5000', '20,260.0000', '20,818.9597', '566.8800']), ('0005', ['คณะสหเวชศาสตร\uf70eสาขาวชาเทคนคการแพทย', '423', '111', '111', '22,234.8500', '19,511.1500', '20,073.2225', '440.9015']), ('0006', ['คณะสหเวชศาสตร\uf70eสาขาวชากายภาพบาบด', '366', '72', '72', '21,661.9

In [122]:
import csv
def read_major_title(filename):
    majors = []
    reader = csv.reader(open(filename))
    for row in reader:
        majors.append(row)
    return majors

majors = read_major_title(major_filename)
print(majors[:3])

[['0001', 'คณะทันตแพทยศาสตร์', '0001', 'จุฬาลงกรณ์มหาวิทยาลัย'], ['0002', 'คณะเภสัชศาสตร์ สาขาวิชาเภสัชศาสตร์', '0001', 'จุฬาลงกรณ์มหาวิทยาลัย'], ['0003', 'คณะเภสัชศาสตร์ สาขาวิชาการบริบาลทางเภสัชกรรม', '0001', 'จุฬาลงกรณ์มหาวิทยาลัย']]


In [123]:
result_row = 0
results = []
for mdata in majors:
    number = mdata[0]
    org_result_row = result_row
    while (result_row < len(pre_results)) and (pre_results[result_row][0] != number):
        result_row += 1
    if result_row == len(pre_results):
        result_row = org_result_row
        continue
    results.append((mdata, pre_results[result_row]))
print(results[:3])

[(['0001', 'คณะทันตแพทยศาสตร์', '0001', 'จุฬาลงกรณ์มหาวิทยาลัย'], ('0001', ['คณะทนตแพทยศาสตร', '329', '29', '29', '23,640.7000', '22,437.4000', '22,810.9724', '337.2389'])), (['0002', 'คณะเภสัชศาสตร์ สาขาวิชาเภสัชศาสตร์', '0001', 'จุฬาลงกรณ์มหาวิทยาลัย'], ('0002', ['คณะเภสชศาสตร\uf70eสาขาวชาเภสชศาสตร', '553', '100', '100', '23,024.4500', '20,221.5500', '20,764.8900', '493.8322'])), (['0003', 'คณะเภสัชศาสตร์ สาขาวิชาการบริบาลทางเภสัชกรรม', '0001', 'จุฬาลงกรณ์มหาวิทยาลัย'], ('0003', ['คณะเภสชศาสตร\uf70eสาขาวชาการบรบาลทางเภสชกรรม', '454', '90', '90', '21,661.2500', '20,129.4500', '20,600.7872', '396.1231']))]


In [124]:
sd_pattern = r'[0-9,]+\.\d+'

def extract_at(data, loc):
    if re.match(r'\d+',data[loc - 6]):
        if loc != -1:
            return data[loc - 6:loc + 1]
        else:
            return data[loc - 6:]
    else:
        if loc != -1:
            return data[loc - 4:loc + 1]
        else:
            return data[loc - 6:]

def extract_zero_row(data):
    if data[-1] == '0':
        return data[-3:]
    try:
        i = -1
        while data[i] != '0':
            i -= 1
        return data[i-2:i+1]
    except:
        print('Error:',data)
        return data

def extract_scores(data):
    if re.match(sd_pattern,data[-1]):
        return extract_at(data, -1)
    else:
        i = -1
        try:
            while not re.match(sd_pattern,data[i]):
                i -= 1
            return extract_at(data, i)
        except:
            return extract_zero_row(data)
        
            
def extract_line(res):
    title = res[0]
    data = res[1][1]
    items = [title[0], title[1], title[3]] + extract_scores(data)
    return items

def print_line(res):
    print(",".join(extract_line(res)))

In [125]:
for r in results[105:115]:
    print_line(r)

0106,คณะวนศาสตร์ สาขาวิชาเทคโนโลยีเยื่อและกระดาษ,มหาวิทยาลัยเกษตรศาสตร์,84,20,20,17,236.9500,13,322.2000,14,344.7550,759.6839
0107,คณะวิทยาศาสตร์ สาขาวิชาคณิตศาสตร์,มหาวิทยาลัยเกษตรศาสตร์,91,28,28,17,550.0500,14,385.6000,15,510.8411,801.5337
0108,คณะวิทยาศาสตร์ สาขาวิชาเคมี,มหาวิทยาลัยเกษตรศาสตร์,167,44,44,18,175.7000,15,765.8500,16,383.9795,512.9524
0109,คณะวิทยาศาสตร์ สาขาวิชาเคมีอุตสาหกรรม,มหาวิทยาลัยเกษตรศาสตร์,70,20,20,17,130.0500,15,198.9500,15,903.4925,597.4402
0110,คณะวิทยาศาสตร์ สาขาวิชาชีววิทยา,มหาวิทยาลัยเกษตรศาสตร์,157,40,40,17,809.4500,14,490.4000,15,585.0350,775.1115
0111,คณะวิทยาศาสตร์ สาขาวิชาสัตววิทยา,มหาวิทยาลัยเกษตรศาสตร์,112,40,40,17,943.1000,13,778.2500,15,027.1625,949.7622
0112,คณะวิทยาศาสตร์ สาขาวิชาพฤกษศาสตร์,มหาวิทยาลัยเกษตรศาสตร์,93,30,30,15,728.2000,11,094.6000,13,494.5900,1,276.2072
0113,คณะวิทยาศาสตร์ สาขาวิชาพันธุศาสตร์,มหาวิทยาลัยเกษตรศาสตร์,540,60,60,20,641.3500,14,449.4500,15,302.3192,986.1432
0114,คณะวิทยาศาสตร์ สาขาวิชาจุลชีววิทยา,มหาวิทยาลัยเกษตรศาสต

In [126]:
def save_result(results, filename):
    header = ['รหัส','คณะ/สาขา','มหาวิทยาลัย','สมัคร','รับ','ผ่าน','สูงสุด','ต่ำสุด','เฉลี่ย','SD']
    writer = csv.writer(open(filename,'w'))
    writer.writerow(header)
    for r in results:
        writer.writerow(extract_line(r))

In [127]:
save_result(results,output_filename)